<a href="https://colab.research.google.com/github/oda251/mojiokoshi_tools/blob/main/ReasonSpeech_%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **使用方法**
#### 1. 上部メニューの「ランタイム」「ランタイムのタイプを変更」から、「T4 GPU」を選択してください
#### 2. 「ランタイム」から「すべてのセルを実行」をクリックすると、output.txtに結果が格納されます。
#### 2. 上から順番に、指示に従って入力等を行ってください


In [ ]:
#@title 必要なライブラリのインストール
!pip install git+https://github.com/reazon-research/reazonspeech

import torch
from espnet2.bin.asr_inference import Speech2Text
import librosa
import reazonspeech as rs
from google.colab import files
import os

In [ ]:
#@title 音声ファイルのアップロード
#@markdown - 実行後、以下のボタンからアップロードしてください
uploaded = files.upload()
file_path = list(uploaded)[0]
import os
file_name = os.path.splitext(os.path.basename(file_path))[0]

device = "cuda" if torch.cuda.is_available() else "cpu"

speech2text = Speech2Text.from_pretrained(
  "reazon-research/reazonspeech-espnet-next",
  beam_size=5,
  batch_size=0,
  device=device
)
SAMPLE_RATE = 13000
speech, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

In [ ]:
#@title 実行
def create_file(file_name="transcribe", file_type="srt"):
    with open(f"{file_name}.{file_type}", mode="w", encoding="utf-8") as f:
        for index, _dict in enumerate(rs.transcribe(speech, speech2text)):
            start_time = _dict.start_seconds
            end_time = _dict.end_seconds
            text = _dict.text
            s_h, e_h = int(start_time//(60 * 60)), int(end_time//(60 * 60))
            s_m, e_m = int((start_time % (60 * 60))//60), int((end_time % (60 * 60))//60)
            s_s, e_s = int(start_time % 60), int(end_time % 60)
            if file_type=="srt":
                f.write(f'{index+1}\n{s_h:02}:{s_m:02}:{s_s:02},000 --> {e_h:02}:{e_m:02}:{e_s:02},000\n{text}\n\n')
            elif file_type=="txt":
                f.write(f'[{s_h:02}:{s_m:02}:{s_s:02} ===> {e_h:02}:{e_m:02}:{e_s:02}] {text}\n')

#@markdown - 出力先ファイルの拡張子を選択してください
file_type = "srt" #@param ["srt", "txt"]
create_file(file_name=file_name , file_type=file_type)

print(f"ファイルが保存されました: {file_name}.{file_type}")